<a href="https://colab.research.google.com/github/Rohit909-creator/Rohit909-creator/blob/main/Sara_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SARA for IEEE SCT SB

As if now this 3rd version of Sara knows to chat,I got to train her with the action_pred_data for action prediction and also a task classification layer for both conversation and app action performance

Note: Action prediction has been done by SARA 2.0 but since a transformer decoder layer is better than LSTM's SARA 3.0 is being made

In [ ]:
!pip install simpletransformers -q

     |████████████████████████████████| 249 kB 24.1 MB/s 
     |████████████████████████████████| 4.7 MB 47.1 MB/s 
     |████████████████████████████████| 1.8 MB 48.1 MB/s 
     |████████████████████████████████| 9.1 MB 41.9 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 6.6 MB 37.5 MB/s 
     |████████████████████████████████| 365 kB 25.9 MB/s 
     |████████████████████████████████| 1.3 MB 45.8 MB/s 
     |████████████████████████████████| 120 kB 9.8 MB/s 
     |████████████████████████████████| 181 kB 24.1 MB/s 
     |████████████████████████████████| 158 kB 46.5 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 157 kB 66.5 MB/s 
     |████████████████████████████████| 157 kB 69.4 MB/s 
     |████████████████████████████████| 157 kB 72.3 MB/s 
     |████████████████████████████████| 157 kB 56.7 MB/s 
     |████████████████████████████████| 157 kB 69.2 MB/s 
     |█████████████

In [ ]:
import torch
import torch.nn as nn
from simpletransformers.language_representation import RepresentationModel
import numpy as np
import pandas as ps
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


In [ ]:
intents = {'intents':[{'tag':'greetings','patterns':['hello','hi','hola','namaskar'],'responses':['hello','hi','how can i help']},
                      {'tag':'actions','patterns':['get me to pdf','take me to pdf','open pdf',
                      'go to assignments','take me to assignments','open assignments','take me to voice notes'
                      ,'open voice notes','take me to voice notes','open voice notes','go back','open recent','i would like to view my assignments','i want to access the assignments page',
                      'i want to get to the assignments page','Take me to chapter 1 of assignments','Take me to chapter 1 in my voice notes','Open my voice notes and lead me to chapter 1',
                      'Take me to chapter 1 of the pdf','I would like to open the pdf and go to chapter 1','Let me see chapter 1 of the PDF'],'responses':['ok','here you go','how can i help']},

                                                                                
                      {'tag':'thanks','patterns':['thank you','thanks'],'responses':['you are welcome','happy to help']},
                      {'tag':'read','patterns':['how many left to complete in pdf','how many left New in pdf','how many have I completed in pdf','how many assignments remain to be completed','what are the remaining assignments','the number of assignments left to complete','is there any work left to do'],'responses':['read']},
                      {'tag':'bye','patterns':['bye bye','ta ta','see ya','shutdown'],'responses':['bye bye']},
                      {'tag':'wishing','patterns':['how are you','you good','how are you feeling','do you feel well','good to hear from you','what is going on with you',"what's up with you",'you seem to be having a hard time'],'responses':['I am good',"I am fine",'Yeah I am good']},
                      {'tag':'sort','patterns':['arange this in datewise order','sort these in datewise order','arrange these pending wise order'],'responses':['sorting,,,']},
                      {'tag':'bookmark','patterns':['bookmark this page','save this page','note this page for later','keep this page handy'],'responses':['bookmarking']},
                      {'tag':'maker','patterns':['who created you','who is your maker','the name of your father',''],'responses':['I am SARA I originless']},
                      {'tag':'age','patterns':['how old are you','what is your age'],'responses':['I dont have an age as of now',"is it nwccessary to have that one"]},
                      {'tag':'place','patterns':['where do you live','where is your place'],'responses':['Look behind you',"I am watching you","Look around you"]},
                      {'tag':'locate','patterns':['find me','locate me'],'responses':['I found you',"gotcha"]},
                      {'tag':'user identity','patterns':['who am i','do you know me','say my name','what is my name'],'responses':['you are <user_name>','i know who you are , you are <user_name>']},
                      {'tag':'speak up','patterns':['say something','Make a statement','Get your point across','Speak up'],'responses':['I like bread','Blah Blah Blah']},
                      {'tag':'story','patterns':['tell me story','How about a story?','A story would be great'],'responses':['<story>']},
                      {'tag':'skills','patterns':['how can you help me','Is there anything you can do?','Are you able to assist?'],'responses':['I can chat and make you feel comfortable , and I can also navigate the windows for you']},
                      {'tag':'status','patterns':['will you marry me','do you have a boyfriend','Are you single'],'responses':['Sorry I cannot be in a relationship','I am an AI and my responsibility to assist you in this app','Sorry I cannot be in a relationship']},
                      {'tag':'other_bots','patterns':['do you know other chatbots','do you know alexa','Are you friends with alexa and google'],'responses':['Yes I do know them','Yes I do know other chatbots']},
                      {'tag':'appreciated','patterns':['you are awesome','you are amazing','you are so sweet','you are beautiful','There is nothing like you','You are an amazing person'],'responses':['Oh thank you thats so sweet','Thank you buddy','Awwww thank you']},
                      {'tag':'demoted','patterns':['i hate you','There is something wrong with you','My opinion of you is negative','Your behavior does not please me','you are not help full','that was stupid','that was not the right answer'],'responses':['I am sorry for the troubles','I will fix myself','Sorry I will fix myself']},
                      {'tag':'google','patterns':['google who is the','who is Ironman','who is the father of the nation india','search this '],'responses':['<search>']},
                      {'tag':'sleep','patterns':['can you sleep','do you sleep','what do you do at night','Have you ever slept','How much sleep do you get'],'responses':['i dont sleep cause I have to help you children with your studies','I cannot sleep and I wont']},
                      {'tag':'color','patterns':['what is your favourite color','which color do you like','any favourite colors'],'responses':['I like all colours','I love all the colors']},
                      {'tag':'food','patterns':['what is your favourite food','which food do you like','what do you like to eat'],'responses':['I like electricity','I like to eat batteries']},
                      {'tag':'play','patterns':['do you play football','do you do sports','do you play cricket'],'responses':['I would love to but I cant','I would love to play but I cant']},
                      
                      
                      
                      
                      ]}

tags = []

for intent in intents['intents']:
    tags.append(intent['tag'])




sentences = []
for intent in intents['intents']:
  for pattern in intent['patterns']:
    sentences.append(pattern)
# print(sentences)
# print(len(sentences))
targets = []
for i,intent in enumerate(intents['intents']):
  pattern = intent['patterns']
  for s in pattern:
    targets.append(i)

target_data = []

for i in targets[0:len(targets)]:
  d = [0]*len(tags)
  d[i] = i
  target_data.append(d)
print(targets)
# print(target_data[0:5])
labels = torch.tensor(target_data,dtype = torch.float32)
print(labels[0:5])    
print(labels[20:30])

In [ ]:
tokenizer = Tokenizer(num_words=116,oov_token='oov')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(['hello how are ya'])



In [ ]:
print(word_index)

{'oov': 1, 'you': 2, 'to': 3, 'do': 4, 'me': 5, 'is': 6, 'are': 7, 'the': 8, 'assignments': 9, 'how': 10, 'pdf': 11, 'what': 12, 'your': 13, 'take': 14, 'open': 15, 'i': 16, 'of': 17, 'voice': 18, 'notes': 19, 'like': 20, 'my': 21, 'page': 22, 'chapter': 23, '1': 24, 'in': 25, 'this': 26, 'who': 27, 'a': 28, 'get': 29, 'many': 30, 'left': 31, 'there': 32, 'with': 33, 'go': 34, 'would': 35, 'and': 36, 'have': 37, 'be': 38, 'order': 39, 'name': 40, 'know': 41, 'story': 42, 'can': 43, 'not': 44, 'sleep': 45, 'favourite': 46, 'want': 47, 'see': 48, 'complete': 49, 'completed': 50, 'any': 51, 'bye': 52, 'ta': 53, 'good': 54, 'up': 55, 'datewise': 56, 'these': 57, 'father': 58, 'where': 59, 'say': 60, 'something': 61, 'help': 62, 'alexa': 63, 'google': 64, 'amazing': 65, 'that': 66, 'was': 67, 'color': 68, 'which': 69, 'food': 70, 'play': 71, 'hello': 72, 'hi': 73, 'hola': 74, 'namaskar': 75, 'back': 76, 'recent': 77, 'view': 78, 'access': 79, 'lead': 80, 'let': 81, 'thank': 82, 'thanks': 83

In [ ]:
class Self_Attention(nn.Module):


  def __init__(self,dim_size,embed_size):
    super().__init__()
    self.dim_size = dim_size
    self.queries = nn.Linear(dim_size,embed_size)
    self.keys = nn.Linear(dim_size,embed_size)
    self.values = nn.Linear(dim_size,embed_size)
    self.softmax = nn.Softmax(dim = 0)
  

  def forward(self,Query,Keys):

    Q = self.queries(X)
    K = self.keys(X)
    V = self.values(X)
    K = K.t()
    attn_scores = torch.matmul(Q,K)
    softmaxed_attn_scores = self.softmax(attn_scores)
    out = torch.matmul(softmaxed_attn_scores,V)

    return out

    



class Multi_headed_attention(nn.Module):

    def __init__(self,embeddings,head_dim,num_heads = 2):
        super().__init__()

        self.head_dim = head_dim
        self.num_heads = num_heads

        self.queries = nn.Linear(embeddings,num_heads*head_dim)
        self.keys = nn.Linear(embeddings,num_heads*head_dim)
        self.values = nn.Linear(embeddings,num_heads*head_dim)
        self.softmax = nn.Softmax(dim = -1)
        self.unify = nn.Linear(num_heads*head_dim,head_dim)

    def forward(self,X):
        bs,seq,embed_dim = X.shape
        Q = self.queries(X).view(bs,seq,self.num_heads,self.head_dim).transpose(1,2)
        K = self.keys(X).view(bs,seq,self.num_heads,self.head_dim).transpose(1,2)
        V = self.values(X).view(bs,seq,self.num_heads,self.head_dim).transpose(1,2)
        K = K.transpose(-1,-2)
        attn_scores = torch.matmul(Q,K)/(self.head_dim**(1/float(2)))
        softmaxed_attn_scores = self.softmax(attn_scores)
        out = torch.matmul(softmaxed_attn_scores,V)
        out = out.transpose(1,2).contiguous().view(bs,seq,self.num_heads*self.head_dim)
        out = self.unify(out)
        return out


class BERT_NN(nn.Module):

  def __init__(self,Self_Attention,encoder,inp_size,embed_size,hidd_size,out_size,action_dim):
    super().__init__()

    self.encoder = encoder
    self.attention = Self_Attention(inp_size,hidd_size)
    self.fc1 = nn.Linear(1792,hidd_size)
    self.fc2 = nn.Linear(hidd_size,hidd_size)
    self.fc3 = nn.Linear(hidd_size,out_size)
    self.action = nn.Linear(hidd_size,action_dim)
    self.relu = nn.ReLU()

  def forward(self,X):
    bs,seq,embeddings = X.shape
    #print(f'bs:{bs}')
    out = self.attention(X)
    out = out.reshape(bs,-1)
    out = self.relu(self.fc1(out))
    out = self.relu(self.fc2(out))
    output = self.fc3(out)
    #action = self,action(out)

    return output,out



In [ ]:
encoder = RepresentationModel(model_type ='bert',model_name='bert-base-uncased',use_cuda = False)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentences2 = sentences[:]
# id(sentences) == id(sentences2)
print(sentences2[23])
sentences2.insert(32,sentences2[23])
sentences2.insert(64,sentences2[23])
sentences2.insert(96,sentences2[23])
sentences2.insert(114,sentences2[23])


print(sentences2[32])
print(sentences2[64])
print(sentences2[96])
print(sentences2[114])


embeddings = encoder.encode_sentences(sentences2[0:32],combine_strategy = None,batch_size=32)
embeddings2 = encoder.encode_sentences(sentences2[32:64],combine_strategy = None,batch_size=32)
embeddings3 = encoder.encode_sentences(sentences2[64:96],combine_strategy = None,batch_size=32)
embeddings4 = encoder.encode_sentences(sentences2[96:-1],combine_strategy = None,batch_size=32)

D = np.append(embeddings,embeddings2[1:] ,axis = 0)
D = np.append(D,embeddings3[1:],axis = 0)
D = np.append(D,embeddings4[1:],axis = 0)

print(len(sentences))
D.shape



I would like to open the pdf and go to chapter 1
I would like to open the pdf and go to chapter 1
I would like to open the pdf and go to chapter 1
I would like to open the pdf and go to chapter 1
I would like to open the pdf and go to chapter 1
114


(114, 14, 768)

In [ ]:
train_data = torch.tensor(D,dtype = torch.float32)

In [ ]:
from torch.utils.data import Dataset,DataLoader,TensorDataset

labels = torch.tensor(target_data,dtype = torch.float32)

dataset = TensorDataset(train_data,labels)

train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

data,t = next(iter(train_dataloader))


In [ ]:
data.shape
t.shape

torch.Size([32, 25])

In [ ]:
import math
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/32)
print(total_samples,n_iterations)

114 4


In [ ]:
num_epochs = 1009
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/32)
model = BERT_NN(Multi_headed_attention,encoder,768,768,128,len(tags),10)
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)
loss = nn.CrossEntropyLoss()

In [ ]:
#training:
model.train()

for epoch in range(num_epochs):
  running_loss = 0.0
  last_loss = 0.0
  for i, data in enumerate(train_dataloader):
          # Every data instance is an input + label pair
          inputs, labels = data

          # Zero your gradients for every batch!
          optimizer.zero_grad()

          # Make predictions for this batch
          out,actions = model(inputs)

          # Compute the loss and its gradients
          l = loss(out, labels)
          l.backward()

          # Adjust learning weights
          optimizer.step()

          # Gather data and report
          running_loss += l.item()
          #print(i)
          if i % 4 == 3:
              last_loss = running_loss / 32 # loss per batch
              print('  batch {} loss: {}'.format(i + 1, last_loss))
              if last_loss < 0.005:
                print("got there")
                break
              tb_x = epoch * len(train_dataloader) + i + 1
              running_loss = 0.

  if last_loss < 0.005:
    break





# for epoch in range(num_epochs):
#   out,out2 =  model(train_data)
#   optimizer.zero_grad()
#   l = loss(out,labels)
#   if epoch%100 == 0:
#     print(f"Epoch:{epoch} Loss:{l.item()}")
#   if l.item() < 0.1:
#     print(f"Yay Got there at Epoch:{epoch} Loss:{l.item()}")
#     break

#   l.backward()
#   optimizer.step()



  batch 4 loss: 4.185785114765167
  batch 4 loss: 3.96523517370224
  batch 4 loss: 3.392854928970337
  batch 4 loss: 2.876979887485504
  batch 4 loss: 2.3044702410697937
  batch 4 loss: 1.7979947328567505
  batch 4 loss: 1.2964810729026794
  batch 4 loss: 0.8488833457231522
  batch 4 loss: 0.5033857673406601
  batch 4 loss: 0.35930079221725464
  batch 4 loss: 0.28984886035323143
  batch 4 loss: 0.1921531818807125
  batch 4 loss: 0.16778169013559818
  batch 4 loss: 0.3519432214088738
  batch 4 loss: 0.13809882197529078
  batch 4 loss: 0.09332390734925866
  batch 4 loss: 0.47508777771145105
  batch 4 loss: 0.2757479501888156
  batch 4 loss: 0.16909528290852904
  batch 4 loss: 0.07426077127456665
  batch 4 loss: 0.05272248084656894
  batch 4 loss: 0.0319086741656065
  batch 4 loss: 0.11038942402228713
  batch 4 loss: 0.047761273104697466
  batch 4 loss: 0.027006940334104
  batch 4 loss: 0.019524297560565174
  batch 4 loss: 0.02129159838659689
  batch 4 loss: 0.012049833836499602
  batch 4

In [ ]:
999%1000

999

# Training:

In [ ]:
l.item()

0.005669111851602793

In [ ]:
model.eval()

BERT_NN(
  (attention): Multi_headed_attention(
    (queries): Linear(in_features=768, out_features=256, bias=True)
    (keys): Linear(in_features=768, out_features=256, bias=True)
    (values): Linear(in_features=768, out_features=256, bias=True)
    (softmax): Softmax(dim=-1)
    (unify): Linear(in_features=256, out_features=128, bias=True)
  )
  (fc1): Linear(in_features=1792, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=25, bias=True)
  (action): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
)

In [ ]:
key = 0
for i,s in enumerate(sentences):
  words = s.split(' ')
  if len(words) + 2 == 14:
    key = (s,len(words),i)

print(key)


('I would like to open the pdf and go to chapter 1', 12, 23)


In [ ]:
test_data = encoder.encode_sentences(['how old are you','which is your favourite colour','access pdf ','I would like to open the pdf and go to chapter 1'],combine_strategy=None)
test_data.shape


['I would like to open the pdf and go to chapter 1 None', 'take me to voice notes None', 'take me to pdf None', 'open assignments None', 'open pdf None', 'Take me to chapter 1 of the pdf None', 'Take me to chapter 1 of assignments None', 'Take me to chapter 1 in my voice notes None', 'go to assignments and then go to pdf None', 'go to pdf and then go to assigments None', 'go to assignments and then go to voice notes None', 'open pdf None', 'go to assignments and then go to pdf None', 'open voice notes None', 'open pdf and go to assignments None', 'open pdf and open assignments None', 'open assignments and go to pdf None', 'open assignments and open pdf None', 'go to pdf and go to assignments None', 'open pdf and go to voice notes None', 'open voice notes and open assignments None', 'open assignments and go to voice notes None', 'open assignments and open voice notes None', 'go to pdf and go to voice notes None']


In [ ]:
test_data = torch.tensor(test_data,dtype = torch.float32)

out,actions = model(test_data)
for i in range(test_data.shape[0]):
  print(tags[torch.argmax(out[i],0).item()])
  print(torch.argmax(out[i],0).item())
  


age
9
color
22
actions
1
play
24


In [ ]:
# checking if the model is overfitting

# test_data_t = torch.tensor(test_data,dtype = torch.float32)

out,actions = model(train_data)
for i in range(train_data.shape[0]):
  print(tags[torch.argmax(out[i],0).item()],tags[targets[i]])
  


In [ ]:
l.item()

0.005616044159978628

In [ ]:
action_data = {'actions':[],'conversation':[]}

for intent in intents['intents']:
  if intent['tag'] in ['actions','bookmark','read','sort','google']:
    action_data['actions'].extend(intent['patterns'])
  else:
    action_data['conversation'].extend(intent['patterns'])

print(action_data)

{'actions': ['get me to pdf', 'take me to pdf', 'open pdf', 'go to assignments', 'take me to assignments', 'open assignments', 'take me to voice notes', 'open voice notes', 'take me to voice notes', 'open voice notes', 'go back', 'open recent', 'i would like to view my assignments', 'i want to access the assignments page', 'i want to get to the assignments page', 'Take me to chapter 1 of assignments', 'Take me to chapter 1 in my voice notes', 'Open my voice notes and lead me to chapter 1', 'Take me to chapter 1 of the pdf', 'I would like to open the pdf and go to chapter 1', 'Let me see chapter 1 of the PDF', 'how many left to complete in pdf', 'how many left New in pdf', 'how many have I completed in pdf', 'how many assignments remain to be completed', 'what are the remaining assignments', 'the number of assignments left to complete', 'is there any work left to do', 'arange this in datewise order', 'sort these in datewise order', 'arrange these pending wise order', 'bookmark this page

In [ ]:
print(tags)

['greetings', 'actions', 'thanks', 'read', 'bye', 'wishing', 'sort', 'bookmark', 'maker', 'age', 'place', 'locate', 'user identity', 'speak up', 'story', 'skills', 'status', 'other_bots', 'appreciated', 'demoted', 'google', 'sleep', 'color', 'food', 'play']


In [ ]:
sorted(action_data['actions'],reverse=True)

In [ ]:
train_data_a_t = ['I would like to open the pdf and go to chapter 1','take me to voice notes','take me to pdf','open assignments','open pdf',
                  'Take me to chapter 1 of the pdf','Take me to chapter 1 of assignments','Take me to chapter 1 in my voice notes','go to assignments and then go to pdf',
                  'go to pdf and then go to assigments','go to assignments and then go to voice notes','open pdf','go to assignments and then go to pdf','open voice notes',
                  'open pdf and go to assignments','open pdf and open assignments','open assignments and go to pdf','open assignments and open pdf',
                  'go to pdf and go to assignments']
print(len(train_data_a_t))                                                                                                                                                                        

# assignments  pdf  voice notes chapter
#this labels dont make sense gotta make an extra class for end of actions
action_target_data = torch.tensor([

                            [[0,0,2,0,0],#1
                             [0,0,0,0,4],
                             ],

                            [[0,0,0,3,0],#2
                             [0,0,0,0,0],
                             ],
                             
                            [[0,0,2,0,0],#3
                             [0,0,0,0,0],
                             ],

                            [[0,1,0,0,0],#4
                             [0,0,0,0,0],
                             ],

                            [[0,0,2,0,0],#5
                             [0,0,0,0,0],
                             ],

                            [[0,0,2,0,0],
                             [0,0,0,0,4],#6
                             ],

                            [[0,1,0,0,0],
                             [0,0,0,0,4],#7
                             ],

                            [[0,0,0,3,0],#8
                             [0,0,0,0,4],
                             ],

                            [[0,1,0,0,0],#9
                             [0,0,2,0,0],
                             ],
                             
                            [[0,0,2,0,0],
                             [0,1,0,0,0],#10
                             ],

                            [[0,1,0,0,0],
                             [0,0,0,3,0],#11
                             ],
                            
                             [[0,0,2,0,0],
                             [0,0,0,0,0],#12
                             ],
                            
                            [[0,1,0,0,0],#13
                             [0,0,2,0,0],
                             ],

                            [[0,0,0,3,0],
                             [0,0,0,0,0],#14
                             ],

                            [[0,0,2,0,0],
                             [0,1,0,0,0],#15
                             ],

                            [[0,0,2,0,0],
                             [0,1,0,0,0],#16
                             ],

                            [[0,1,0,0,0],
                             [0,0,2,0,0],#17
                             ],

                            [[0,1,0,0,0],
                             [0,0,2,0,0],#18
                             ],

                            [[0,0,2,0,0],
                             [0,1,0,0,0],#19
                             ],

                            [[0,0,2,0,0],
                             [0,0,0,3,0],#20
                             ],

                            [[0,0,0,3,0],
                             [0,1,0,0,0],#21
                             ],

                             [[0,1,0,0,0],
                             [0,0,0,3,0],#22
                             ],

                            [[0,1,0,0,0],
                             [0,0,0,3,0],#23
                             ],

                            [[0,0,2,0,0],
                             [0,0,0,3,0],#22
                             ],


                             
],dtype = torch.float32)


print(action_target_data.shape)


19
torch.Size([24, 2, 5])


In [ ]:
train_data_a_t = ['I would like to open the pdf and go to chapter 1','take me to voice notes','take me to pdf','open assignments','open pdf','Take me to chapter 1 of the pdf','Take me to chapter 1 of assignments','Take me to chapter 1 in my voice notes','go to assignments and then go to pdf','go to pdf and then go to assigments','go to assignments and then go to voice notes','open pdf','go to assignments and then go to pdf','open voice notes','open pdf and go to assignments','open pdf and open assignments','open assignments and go to pdf','open assignments and open pdf','go to pdf and go to assignments','open pdf and go to voice notes','open voice notes and open assignments','open assignments and go to voice notes','open assignments and open voice notes','go to pdf and go to voice notes']
train_data_a_t

print(train_data_a_t) 
# 'open pdf and go to voice notes',
#  'open voice notes and open assignments',
#  'open assignments and go to voice notes',
#  'open assignments and open voice notes',
#  'go to pdf and go to voice notes']

['I would like to open the pdf and go to chapter 1', 'take me to voice notes', 'take me to pdf', 'open assignments', 'open pdf', 'Take me to chapter 1 of the pdf', 'Take me to chapter 1 of assignments', 'Take me to chapter 1 in my voice notes', 'go to assignments and then go to pdf', 'go to pdf and then go to assigments', 'go to assignments and then go to voice notes', 'open pdf', 'go to assignments and then go to pdf', 'open voice notes', 'open pdf and go to assignments', 'open pdf and open assignments', 'open assignments and go to pdf', 'open assignments and open pdf', 'go to pdf and go to assignments', 'open pdf and go to voice notes', 'open voice notes and open assignments', 'open assignments and go to voice notes', 'open assignments and open voice notes', 'go to pdf and go to voice notes']


In [ ]:
train_data_a = encoder.encode_sentences(train_data_a_t,combine_strategy=None)
train_data_a.shape

(24, 15, 768)

In [ ]:
l = 0
key = 0
for i,s in enumerate(train_data_a_t):
  words = s.split(' ')
  if len(words) > l:
    key = (s,len(words),i)
    l = len(words)
    print(l)
print(key)


12
('I would like to open the pdf and go to chapter 1', 12, 0)


In [ ]:
class Decoder(nn.Module):

  def __init__(self,inp_size,hid_size,out_size,num_layers):
    super().__init__()
    self.inp_size = inp_size
    self.hid_size = hid_size
    self.out_size = out_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(inp_size,hid_size,num_layers = num_layers,batch_first = True)
    self.out = nn.Linear(896,out_size)
    self.relu = nn.ReLU()

  def forward(self,X,hidden,cell):
    bs,seq_len,hid_size = X.shape
    #for i in range(3):
    out,(hidden,cell) = self.lstm(X,(hidden,cell))
    out = out.reshape(bs,2,-1)
    #print("yay")
    #print(out.shape)
    out = self.relu(self.out(out))
    #print("yay2")
    return out,hidden,cell

  def init_hidden(self):
    
    hidden = torch.zeros((1,24,self.hid_size))
    cell = torch.zeros((1,24,self.hid_size))

    return hidden,cell


class BERT_NN2(nn.Module):

  def __init__(self,Self_Attention,inp_size,embed_size,hidd_size,out_size,action_dim):
    super().__init__()

    self.encoder = encoder
    self.attention = Self_Attention(inp_size,hidd_size)
    self.fc1 = nn.Linear(1792,hidd_size)
    self.fc2 = nn.Linear(hidd_size,hidd_size)
    self.fc3 = nn.Linear(hidd_size,out_size)
    self.action = nn.Linear(hidd_size,action_dim)
    self.relu = nn.ReLU()

  def forward(self,X):
    bs,seq,embeddings = X.shape
    #print(f'bs:{bs}')
    out = self.attention(X)
    out = out.reshape(bs,2,-1)
    out = self.relu(self.fc1(out))
    out = self.relu(self.fc2(out))
    output = self.fc3(out)
    #action = self,action(out)

    return output



In [ ]:
model = BERT_NN2(Multi_headed_attention,768,768,128,5,5)

#Decoder

In [ ]:
model.eval()

BERT_NN(
  (attention): Multi_headed_attention(
    (queries): Linear(in_features=768, out_features=256, bias=True)
    (keys): Linear(in_features=768, out_features=256, bias=True)
    (values): Linear(in_features=768, out_features=256, bias=True)
    (softmax): Softmax(dim=-1)
    (unify): Linear(in_features=256, out_features=128, bias=True)
  )
  (fc1): Linear(in_features=1792, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=25, bias=True)
  (action): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
)

In [ ]:
model.attention.requires_grad_ = False
model.fc1.requires_grad_ = False
model.fc2.requires_grad_ = False
model.fc3.requires_grad_ = False

train_data_a = encoder.encode_sentences(train_data_a_t,combine_strategy=None)
train_data_a.shape

(24, 14, 768)

In [ ]:
decoder = Decoder(768,128,5,1)

decoder.train()

# optimizer2 = torch.optim.AdamW(decoder.parameters())
# loss2 = nn.CrossEntropyLoss()
train_data = torch.tensor(train_data_a.tolist(),dtype = torch.float32)
# T,actions = model(train_data)
# actions.requires_grad_ = False
# actions = actions.detach()
# print(actions.shape)
hidden,cell = decoder.init_hidden()
out,hidden,cell = decoder(train_data,hidden,cell)
print(out.shape)
# print(out)

torch.Size([24, 2, 5])


In [ ]:
decoder = Decoder(768,128,5,1)
num_epochs = 1009
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(decoder.parameters(),lr = 0.0001)


In [ ]:

for epochs in range(num_epochs):
  hidden,cell = decoder.init_hidden()
  out,hidden,cell = decoder(train_data,hidden,cell)
  # out = model2(actions)
  l = loss(out,action_target_data)
  optimizer.zero_grad()
  if epochs%100 == 0:
    print(f"At Epoch:{epochs} Loss:{l.item()}")
    
  if l.item() < 0.09:
    print(f"YAY got there at Epoch:{epoch} Loss:{l.item()}")
    break
  l.backward()
  optimizer.step()


At Epoch:0 Loss:0.5079899430274963
YAY got there at Epoch:37 Loss:0.0894903689622879


In [ ]:
print(l.item())

0.0894903689622879


In [ ]:
decoder.eval()
# model2.eval()

Decoder(
  (lstm): LSTM(768, 128, batch_first=True)
  (out): Linear(in_features=896, out_features=5, bias=True)
  (relu): ReLU()
)

In [ ]:
test_data = encoder.encode_sentences(['open assignments','go to assignments and then go to pdf ','I would like to open the pdf and go to voice notes'],combine_strategy=None)
test_data = torch.tensor(test_data.tolist(),dtype = torch.float32)
# out,actions = model(test_data)
# action_output = model2(actions)
hidden,cell = torch.zeros((1,3,128)),torch.zeros((1,3,128))
action_output,hidden,cell = decoder(test_data,hidden,cell)
#print(action_output)

In [ ]:
action = ["None","assignments","pdf","voice notes","chapter"]

In [ ]:

T = torch.argmax(action_output[0],dim = 1)
print(action[T[0].item()],action[T[1].item()])

T = torch.argmax(action_output[1],dim = 1)
print(action[T[0].item()],action[T[1].item()])

T = torch.argmax(action_output[2],dim = 1)
print(action[T[0].item()],action[T[1].item()])


assignments chapter
assignments voice notes
pdf chapter


In [ ]:
dataset = TensorDataset(train_data,action_target_data)
train_dataloader = DataLoader(dataset,batch_size=10)
d = enumerate(train_dataloader)
i,d = next(d)
print(i,d[0].shape,d[1].shape)


# for epoch in range(num_epochs):
#   running_loss = 0.0
#   last_loss = 0.0
#   # print('1')
#   for i, data in enumerate(train_dataloader):
#           # Every data instance is an input + label pair
#           inputs, labels = data

#           # Zero your gradients for every batch!
#           optimizer.zero_grad()

#           # Make predictions for this batch
#           out = decoder(inputs)

#           # Compute the loss and its gradients
#           l = loss(out, labels)
#           l.backward()
#           # print(l.item())
#           # Adjust learning weights
#           optimizer.step()

#           # Gather data and report
#           running_loss += l.item()
#           #print(i)
#           if i % 3 == 2:
#             last_loss = running_loss / 10 # loss per batch
#             print(last_loss)
#             print('  batch {} loss: {}'.format(i + 1, last_loss))
#             if last_loss < 0.01:
#               print("got there")
#               break
#             tb_x = epoch * len(train_dataloader) + i + 1
#             running_loss = 0.

#   if last_loss < 0.01:
#     print('2')
#     break



0 torch.Size([10, 14, 768]) torch.Size([10, 2, 5])


In [ ]:
import math
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/10)
print(total_samples,n_iterations)


24 3
